<a href="https://colab.research.google.com/github/sssangeetha/OutamationAI_OCR_RAG_Automation/blob/main/Full_RAG_Chatbot_Gradio_OpenSource.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# System packages (quiet)
!apt-get -qq update
!apt-get -qq install -y tesseract-ocr libtesseract-dev poppler-utils >/dev/null


W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)


In [2]:
!pip -q install pytesseract pdf2image pypdf \
  sentence-transformers faiss-cpu \
  langchain langchain-community \
  transformers accelerate gradio tiktoken
# Optional (for better ranking). Comment out if you want it faster to install:
!pip -q install "sentence-transformers[cross-encoder]"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.9/323.9 kB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 75.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 101.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.


In [2]:
THEME_CSS = """
:root { --brand: #5b8cff; --bg: #0b0f19; --card: #12182a; --text: #e8eefc; --muted: #9db0d8; }
.gradio-container { background: var(--bg); color: var(--text); }
h1, h2, h3 { color: var(--text); }
button { border-radius: 12px; }
input, textarea { background: var(--card)!important; color: var(--text)!important; border-radius: 10px!important; }
"""
with open("theme.css","w") as f:
    f.write(THEME_CSS)


In [3]:
import os, io, time, re, uuid, json
from typing import List, Dict, Any, Tuple
from dataclasses import dataclass

import numpy as np
import faiss
import gradio as gr

from pypdf import PdfReader
from pdf2image import convert_from_bytes
import pytesseract

from sentence_transformers import SentenceTransformer
try:
    from sentence_transformers.cross_encoder import CrossEncoder
    HAS_RERANK = True
except Exception:
    HAS_RERANK = False

from langchain.text_splitter import RecursiveCharacterTextSplitter
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

# ---- Config ----
EMBED_MODEL_NAME   = "sentence-transformers/all-MiniLM-L6-v2"
CROSS_ENCODER_MODEL= "cross-encoder/ms-marco-MiniLM-L-6-v2"
LLM_MODEL_NAME     = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"   # swap to "Qwen/Qwen2.5-1.5B-Instruct" if you have GPU
DEVICE_MAP         = "auto"

CHUNK_SIZE     = 900
CHUNK_OVERLAP  = 140
DEFAULT_TOPK   = 4
MAX_NEW_TOKENS = 512
TEMPERATURE    = 0.2

SYSTEM_PROMPT = (
    "You are a helpful RAG assistant. Use ONLY the provided context to answer.\n"
    "If the answer is not in the context, say you do not know.\n"
    "Include citations with [doc_name p:page] for every claim.\n"
    "Be concise, accurate, and neutral.\n"
)


In [4]:
@dataclass
class Chunk:
    text: str
    metadata: Dict[str, Any]

def guess_doc_type(name: str) -> str:
    n = name.lower()
    if "resume" in n or "cv" in n: return "Resume"
    if "contract" in n or "agreement" in n: return "Contract"
    if "invoice" in n or "receipt" in n: return "Invoice"
    if "policy" in n: return "Policy"
    return "Document"

def _slugify(s: str) -> str:
    s = re.sub(r"[^a-zA-Z0-9]+", "-", s)
    return s.strip("-").lower()

def extract_text_from_pdf(pdf_bytes: bytes):
    page_texts, page_meta = [], {}
    reader = PdfReader(io.BytesIO(pdf_bytes))
    digital_texts = []
    for i, page in enumerate(reader.pages):
        try:
            txt = page.extract_text() or ""
        except Exception:
            txt = ""
        digital_texts.append(txt)

    # Use native text if most pages have text
    if sum(1 for t in digital_texts if t.strip()) >= max(1, int(0.6 * len(digital_texts))):
        for i, txt in enumerate(digital_texts):
            page_texts.append(txt.strip())
            page_meta[i] = {"method": "text"}
        return page_texts, page_meta

    # Otherwise: OCR fallback
    images = convert_from_bytes(pdf_bytes, dpi=200)
    for i, img in enumerate(images):
        ocr_text = pytesseract.image_to_string(img)
        page_texts.append(ocr_text.strip())
        page_meta[i] = {"method": "ocr", "dpi": 200}
    return page_texts, page_meta

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=CHUNK_SIZE, chunk_overlap=CHUNK_OVERLAP,
    length_function=len, separators=["\n\n", "\n", ". ", " ", ""]
)

def build_chunks(doc_name: str, page_texts: List[str]) -> List[Chunk]:
    chunks: List[Chunk] = []
    for page_idx, page_text in enumerate(page_texts):
        if not page_text.strip():
            continue
        small = text_splitter.split_text(page_text)
        for i, c in enumerate(small):
            chunks.append(Chunk(
                text=c,
                metadata={
                    "doc_name": doc_name,
                    "page": page_idx + 1,
                    "chunk_id": f"{doc_name}-p{page_idx+1}-c{i+1}",
                    "doc_type": guess_doc_type(doc_name),
                    "source_id": _slugify(doc_name),
                }
            ))
    return chunks


In [5]:
def normalize_scores(scores: List[float]):
    # cosine [-1,1] -> [0,1]
    return [0.5 * (s + 1.0) for s in scores]

class FaissIndex:
    def __init__(self, embed_model_name=EMBED_MODEL_NAME):
        self.model = SentenceTransformer(embed_model_name)
        self.dim = self.model.get_sentence_embedding_dimension()
        self.index = faiss.IndexFlatIP(self.dim)
        self.meta:  List[Dict[str, Any]] = []
        self.texts: List[str] = []

    def add(self, chunks: List[Chunk]):
        if not chunks: return
        texts = [c.text for c in chunks]
        embs = self.model.encode(texts, convert_to_numpy=True, normalize_embeddings=True, show_progress_bar=False).astype(np.float32)
        self.index.add(embs)
        self.meta.extend([c.metadata for c in chunks])
        self.texts.extend(texts)

    def search(self, query: str, top_k: int):
        q = self.model.encode([query], convert_to_numpy=True, normalize_embeddings=True).astype(np.float32)
        D, I = self.index.search(q, top_k)
        out = []
        for idx, score in zip(I[0], D[0]):
            if idx != -1:
                out.append((int(idx), float(score)))
        return out

    def get(self, idx: int):
        return self.texts[idx], self.meta[idx]

class OptionalReranker:
    def __init__(self, enabled=True):
        self.enabled = enabled and HAS_RERANK
        self.model = None
        if self.enabled:
            try:
                self.model = CrossEncoder(CROSS_ENCODER_MODEL)
            except Exception:
                self.enabled = False

    def rerank(self, query: str, candidates, top_k: int):
        if not self.enabled or not candidates:
            return candidates[:top_k]
        pairs = [(query, c[2]) for c in candidates]
        scores = self.model.predict(pairs).tolist()
        ranked = sorted([(c[0], float(scores[i]), c[2], c[3]) for i, c in enumerate(candidates)],
                        key=lambda x: x[1], reverse=True)
        return ranked[:top_k]


In [6]:
def load_llm(model_name=LLM_MODEL_NAME, temperature=TEMPERATURE):
    tok = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
    mdl = AutoModelForCausalLM.from_pretrained(model_name, device_map=DEVICE_MAP, trust_remote_code=True)
    return pipeline("text-generation", model=mdl, tokenizer=tok,
                    max_new_tokens=MAX_NEW_TOKENS, temperature=temperature,
                    do_sample=temperature>0.0, return_full_text=False)

def build_prompt(question: str, contexts: List[Dict[str, Any]]) -> str:
    blocks = []
    for c in contexts:
        m = c["metadata"]
        citation = f"[{m['doc_name']} p:{m['page']}]"
        blocks.append(citation + "\n" + c["text"])
    ctx = "\n\n---\n\n".join(blocks) if blocks else "No context."
    user = f"Question: {question}\n\nContext:\n{ctx}\n\nAnswer with citations."
    return f"<s>[SYSTEM]\n{SYSTEM_PROMPT}\n</s>\n[USER]\n{user}\n[/USER]\n[ASSISTANT]"

class RAGPipeline:
    def __init__(self):
        self.index = FaissIndex()
        self.llm = None
        self.reranker = OptionalReranker(enabled=True)

    def ensure_llm(self):
        if self.llm is None:
            self.llm = load_llm()

    def add_pdf(self, file_bytes: bytes, name: str) -> int:
        pages, _ = extract_text_from_pdf(file_bytes)
        chunks = build_chunks(name, pages)
        before = len(self.index.meta)
        self.index.add(chunks)
        return len(self.index.meta) - before

    def query(self, question: str, top_k: int = DEFAULT_TOPK, doc_type_filter: str = "Any", show_chunks: bool = False):
        if len(self.index.meta) == 0:
            return {"answer": "Please upload and index documents first.", "sources": [], "meta": {}, "chunks": []}

        self.ensure_llm()
        t0 = time.time()

        # Stage 1: vector search (recall)
        vec_res = self.index.search(question, top_k=max(top_k*4, top_k))
        candidates = []
        for idx, score in vec_res:
            txt, md = self.index.get(idx)
            if doc_type_filter != "Any" and md.get("doc_type") != doc_type_filter:
                continue
            candidates.append((idx, score, txt, md))

        # Stage 2: optional rerank (precision)
        final = self.reranker.rerank(question, candidates, top_k) if self.reranker.enabled else candidates[:top_k]

        sims = [c[1] for c in final]
        conf = normalize_scores(sims)
        contexts = [{"text": c[2], "metadata": c[3], "score": conf[i]} for i, c in enumerate(final)]

        prompt = build_prompt(question, contexts)
        gen = self.llm(prompt)[0]["generated_text"]
        latency_ms = int((time.time() - t0) * 1000)

        return {
            "answer": gen.strip(),
            "sources": [{
                "doc_name": c["metadata"]["doc_name"],
                "page": c["metadata"]["page"],
                "chunk_id": c["metadata"]["chunk_id"],
                "doc_type": c["metadata"]["doc_type"],
                "score": round(c["score"], 3)
            } for c in contexts],
            "meta": {
                "chunks_used": len(contexts),
                "avg_confidence": round(float(np.mean(conf)) if conf else 0.0, 3),
                "latency_ms": latency_ms,
                "reranker_enabled": self.reranker.enabled
            },
            "chunks": contexts if show_chunks else []
        }

RAG = RAGPipeline()


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

In [8]:
with open("theme.css", "w") as f:
    f.write(open("theme.css").read())


In [9]:
CSS = open("theme.css","r").read()

with gr.Blocks(css=CSS, title="Investor-Ready RAG UI") as demo:
    gr.Markdown("# 🔎 Investor-Ready RAG Chatbot\nPolished demo with citations, confidence bars, and optional reranking.")

    with gr.Row():
        with gr.Column(scale=1):
            gr.Markdown("### Upload & Index")
            files = gr.File(file_count="multiple", file_types=[".pdf"], label="Upload PDFs")
            ingest_btn = gr.Button("➕ Ingest & Index", variant="primary")
            index_log = gr.Markdown("")
            gr.Markdown("### Settings")
            topk = gr.Slider(1, 10, value=DEFAULT_TOPK, step=1, label="Top-K")
            doc_filter = gr.Dropdown(["Any","Contract","Invoice","Policy","Resume","Document"], value="Any", label="Doc Type Filter")
            show_chunks = gr.Checkbox(value=False, label="Show retrieved chunk previews")
            export_btn = gr.Button("⬇️ Export chat (JSON)")

        with gr.Column(scale=2):
            gr.Markdown("### Chat")
            chat = gr.Chatbot(type="messages", height=420)
            q = gr.Textbox(placeholder="Ask a question… e.g., What late fees are mentioned in my contract?", label="Your question")
            ask_btn = gr.Button("🧠 Ask", variant="primary")
            last_answer = gr.Markdown("")

    def do_ingest(file_list):
        names, total = [], 0
        for f in (file_list or []):
            try:
                data = f.read()
                name = os.path.basename(f.name) if hasattr(f, "name") else f"upload-{uuid.uuid4().hex}.pdf"
                added = RAG.add_pdf(data, name)
                total += added
                names.append(f"• {name}  (+{added} chunks)")
            except Exception as e:
                names.append(f"• {getattr(f,'name','unknown')}: ERROR {e}")
        return f"**Indexed:** {total} chunks\n" + "\n".join(names) if names else "No files uploaded."

    def do_ask(question, k, doc_type, show_chunks_flag, history):
        out = RAG.query(question, top_k=int(k), doc_type_filter=doc_type, show_chunks=bool(show_chunks_flag))
        answer = out["answer"]
        meta = out["meta"]
        # Sources w/ confidence bars
        src_lines = []
        for s in out["sources"]:
            conf_bar = "█" * int(round(s["score"] * 10)) or "░"
            src_lines.append(f"- **{s['doc_name']}** (p:{s['page']}, {s['doc_type']}) — conf: {s['score']} {conf_bar}")
        sources_md = "\n".join(src_lines) or "—"
        header = f"**Chunks:** {meta.get('chunks_used',0)} • **Avg conf:** {meta.get('avg_confidence',0)} • **Latency:** {meta.get('latency_ms',0)} ms • **Rerank:** {meta.get('reranker_enabled',False)}"
        chunk_preview = ""
        if out.get("chunks"):
            for i, c in enumerate(out["chunks"], start=1):
                snippet = (c["text"][:480] + "…") if len(c["text"]) > 480 else c["text"]
                m = c["metadata"]
                chunk_preview += f"\n\n> **[{i}] {m['doc_name']} p:{m['page']}**\n> {snippet}"
        rendered = f"{answer}\n\n---\n**Sources**\n{sources_md}\n\n{header}{chunk_preview}"
        history = (history or []) + [("You: " + question, rendered)]
        return history, "", rendered

    def export_chat(history):
        data = [{"user": u, "assistant": a} for (u, a) in (history or [])]
        return json.dumps(data, indent=2)

    ingest_btn.click(do_ingest, inputs=[files], outputs=[index_log])
    ask_btn.click(do_ask, inputs=[q, topk, doc_filter, show_chunks, chat], outputs=[chat, q, last_answer])
    export_btn.click(export_chat, inputs=[chat], outputs=[last_answer])

demo


Gradio Blocks instance: 3 backend functions
-------------------------------------------
fn_index=0
 inputs:
 |-<gradio.components.file.File object at 0x7f36c0192360>
 outputs:
 |-<gradio.components.markdown.Markdown object at 0x7f3520b81a90>
fn_index=1
 inputs:
 |-<gradio.components.textbox.Textbox object at 0x7f350c584920>
 |-<gradio.components.slider.Slider object at 0x7f350c6b1460>
 |-<gradio.components.dropdown.Dropdown object at 0x7f35208fae10>
 |-<gradio.components.checkbox.Checkbox object at 0x7f3520ce5a60>
 |-<gradio.components.chatbot.Chatbot object at 0x7f350c6b2390>
 outputs:
 |-<gradio.components.chatbot.Chatbot object at 0x7f350c6b2390>
 |-<gradio.components.textbox.Textbox object at 0x7f350c584920>
 |-<gradio.components.markdown.Markdown object at 0x7f3520681970>
fn_index=2
 inputs:
 |-<gradio.components.chatbot.Chatbot object at 0x7f350c6b2390>
 outputs:
 |-<gradio.components.markdown.Markdown object at 0x7f3520681970>

In [11]:
# In Colab use .launch(share=True) to get a public link for demos
demo.launch(share=True)


Rerunning server... use `close()` to stop if you need to change `launch()` parameters.
----
Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://ce1fcbbf103a95d1f3.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
